In [ ]:
#Operaciones algebraicas
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

#filtrado para suavizar los datos
from scipy.signal import savgol_filter

In [ ]:
from prophet import Prophet
# from fbprophet.deagnostics import cross_validation

# Forecasting Demanda Energía (Prophet)

In [ ]:
# Importo el archivos de datos de consumo de energia en la zona este de EE.UU.
df = pd.read_csv(r'Raw_Data/medidor_1.csv')
df.head()

In [ ]:
print(df.shape)
print('\n')
print(df.info())

In [ ]:
# Cambio de nombre en columnas y eliminamos otra
df.rename(columns={'fechahora':'Datetime', 'demanda_activa':'y[kW]'}, inplace = True)
df.drop(columns='terminal', inplace=True)
df.head()

<b>Target_values: "y[kW]"</b>

In [ ]:
#Convierto a tipo DateTimeIndex la columna "Datetime"
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)

df.head()

In [ ]:
# Calcula algunos parámetros estadísticos solo sobre las variables de tipo float/int
df.describe()

## Limpieza de datos

### Eliminación de datos duplicados

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
df.drop_duplicates(subset = 'Datetime', keep = 'last', inplace = True)
df.shape

### Tratamiento de espacios vacios para un grupo de datos continuos

In [ ]:
df_2 = df.set_index('Datetime')
df_2.drop(['2017-08-18 09:15:00'], inplace = True)
df_2

In [ ]:
print(df_2.index.min())
print(df_2.index.max())

In [ ]:
print(f'df_2.index.freq is set to: {df_2.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(df_2.index),
                          end = max(df_2.index),
                          freq = '15min') 
#freq = '15min' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(data_range.difference(df_2.index))
print('\n')
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}')

<i>Lo que ese numero indica es la cantidad de puntos en el tiempo perdidos dentro del conjunto de datos</i>

## Filtro savgol_filter

In [ ]:
# Datos sin filtrar
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_2.index, y=df_2['y[kW]'],
                         mode='lines',
                         name='Datos'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
# Aplica filtro elegido en base a buscar cual mejor se adecua
y_filtered = df_2[["y[kW]"]].apply(savgol_filter,  window_length=5, polyorder=3)

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_2.index,y=df_2['y[kW]'],
                         mode='lines',
                         name='No Filtrada'))
fig.add_trace(go.Scatter(x=y_filtered.index, y=y_filtered['y[kW]'],
                         mode='lines', 
                         name='Filtrada'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.show()

In [ ]:
y_filtered.head()

In [ ]:
y_filtered.info()

## Gráfico de distribución

In [ ]:
features_num = ["y[kW]"]
y_filtered[features_num].hist(figsize=(10,4)); #el ; es para evitar una grafica duplicada